In [1]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar
import pickle

In [2]:
# input
ETF_data = pd.read_csv('Emerging Asia Pacific ETF List (114).csv')

# filter the inception date
ETF_data['Inception'] = pd.to_datetime(ETF_data['Inception'], format="%d/%m/%Y")
date = datetime.datetime(2016, 1, 1, 0, 0)
ETF_data = ETF_data.loc[ETF_data['Inception'] < date,:].reset_index()

# extract symbol and name of each ETF
symbol_list = ETF_data['Symbol']
name_list = ETF_data['ETF Name']

In [56]:
homepage_list = []
for i in symbol_list:
    url = 'https://www.etf.com/'+i+'#overview'
    res = requests.get(url)
    doc = pq(res.text)
    doc.make_links_absolute(res.url)
    homepage_list.append(doc('.field-content.helplink')('a').attr('href'))

In [3]:
# dic = {'symbol':[],'name':[],'addr':[]}
# for i,s in enumerate(symbol_list):
#     dic['symbol'].append(s)
#     dic['name'].append(name_list[i])
#     dic['addr'].append(homepage_list[i])
#     print(s,name_list[i],homepage_list[i])
    
# # Save this dic
# with open("ETF_dic.pkl","wb") as f:
#     pickle.dump(dic,f)
# Use
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [4]:
def get_the_last_day_of_month(start_date,end_date):
    ans = []
    temp = start_date - datetime.timedelta(1)
    year = start_date.year
    month = start_date.month
    
    while (temp < end_date):
        temp = temp + datetime.timedelta(calendar.monthrange(year,month)[1])
        ans.append(temp)
        month += 1
        if(month > 12):
            month = 1
            year += 1
    
    # check for trading day
    for i in range(len(ans)):
        if ans[i].isoweekday()>5:
            ans[i]=ans[i]+datetime.timedelta(days=5-ans[i].isoweekday())
    return ans

In [7]:
last_day_of_month = get_the_last_day_of_month(datetime.datetime(2015,12,31,0,0),datetime.datetime(2018,12,31,0,0))

In [8]:
len(last_day_of_month)

37

In [47]:
# Columbia
def get_Columbia_NAV(symbol,addr,start,last_day_of_month):
    
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    if not os.path.isdir('./data/Columbia'):
        os.mkdir('./data/Columbia')
    
    # crawl    
    if start:
        res = requests.get(addr)
        doc = pq(res.text)
        doc.make_links_absolute(res.url)
        filename = doc('#fundliterature .link-list')('li:nth-child(6)')('a').attr('href')
        filename = filename.replace(' ','%20')
        urllib.request.urlretrieve(filename, "./data/Columbia/columbia.xls")
        print('Download columbia.xls')
        
        # filter the last of the month
        data = pd.read_excel('./data/Columbia/columbia.xls')
        data = data.rename(columns = data.iloc[2]).iloc[3:,:]
        data['Date'] = pd.to_datetime(data['Date'], format="%Y/%m/%d")
        
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        data.to_excel('./data/Columbia/columbia_filter_day.xls')
    
    # get NAV of each symbol from Columbia
    
    data_filter = pd.read_excel('./data/Columbia/columbia_filter_day.xls')
    data_filter = data_filter[data_filter['Fund Symbol'] == symbol]

    data_symbol = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
    data_symbol = data_symbol.merge(data_filter[['Date','NAV']],on='Date',how='left')
    data_symbol.to_excel(f'./data/Columbia/{symbol}.xls')

In [48]:
get_Columbia_NAV(dic['symbol'][0],dic['addr'][0],False,last_day_of_month)

In [62]:
dic['name'][5]

'CSOP MSCI China A International Hedged ETF'

In [58]:
dic['addr'][4]

'http://www.csopasset.us/en-us/products/etf/a50/'

In [ ]:
# CSOP 
# forbidden!!!!!
def get_CSOP_NAV(symbol,addr,start,last_day_of_month):
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    if not os.path.isdir('./data/CSOP'):
        os.mkdir('./data/CSOP')
        
     # crawl    
    if start:
        res = requests.get(addr)
        doc = pq(res.text)
        doc.make_links_absolute(res.url)
        filename = doc('#fundliterature .link-list')('li:nth-child(6)')('a').attr('href')
        filename = filename.replace(' ','%20')
        urllib.request.urlretrieve(filename, "./data/Columbia/columbia.xls")
     


In [91]:
res = requests.get(dic['addr'][4])
doc = pq(res.text)
doc.make_links_absolute(res.url)


[<html>]

In [92]:
res.text

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body bgcolor="white">\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [96]:
from selenium import webdriver

In [105]:
browser=webdriver.Chrome('/Users/lou/Downloads/chromedriver') 
browser.get(dic['addr'][4])
browser.close()

In [106]:
dic['name'][6]

'Direxion Daily China 3x Bear Shares'

In [109]:
dic['addr'][6]

'http://www.direxioninvestments.com/products/direxion-daily-china-bear-3x-etf'

In [107]:
# Direxion
# no historical data for NAV, but there are some free trail to get the data
# https://ycharts.com/companies/YINN
def get_Direxion_NAV(symbol,addr,start,last_day_of_month):
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    if not os.path.isdir('./data/Direxion'):
        os.mkdir('./data/Direxion')

In [118]:
print(dic['name'][13])
print(dic['addr'][13])

Emerging Markets Internet & Ecommerce ETF
https://emqqetf.com/


In [ ]:
# emqq
def get_EMQQ_NAV(symbol,addr,start,last_day_of_month):
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    if not os.path.isdir('./data/EMQQ'):
        os.mkdir('./data/EMQQ')

In [ ]:
dic

### Deal with xml file 

In [63]:
import xml.etree.ElementTree as ET


In [64]:
tree = ET.parse('ck0001617113-20170930.xml')

In [66]:
root = tree.getroot()

In [68]:
root.attrib

{}

In [71]:
from lxml import objectify
xml = objectify.parse('ck0001617113-20170930_lab.xml')
root = xml.getroot()

data=[]
for i in range(len(root.getchildren())):
    data.append([child.text for child in root.getchildren()[i].getchildren()])